In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import xgboost as xgb
from sklearn.model_selection import train_test_split
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import r2_score

# https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/

In [2]:
path1 = 'https://gen10datafund2202.blob.core.windows.net/jedscontainer/911_Calls_For_Service.csv'
path2 = 'https://opendata.arcgis.com/api/v3/datasets/4f49eb825f564efa9a23cd103c4ba13b_0/downloads/data?format=csv&spatialRefId=4326'
start = time.perf_counter()
detroit_911 = pd.read_csv(path1, thousands = ",")#, nrows = 100000)
print(time.perf_counter() - start)

79.1192418


## Clean the Data

In [3]:
# Make a copy of the dataframe so that I don't have to redownload it every time I want to refresh.
detroit_df = detroit_911.copy()

# Count all the null values in the columns
null_values = detroit_911.isnull().sum()
#print('Null values prior to dropping columns.')
#print(null_values)

# Drop columns that will not be used in algorithm
detroit_df.drop(
    columns = [
        'incident_id',
        'zip_code',
        'oid',
        'respondingunit',
        'agency',
        'X',
        'Y',
        'longitude',
        'latitude',
        'incident_address',
        'block_id',
        'category'
    ], inplace = True)

#print('\nNull values after dropping columns')
#print(detroit_df.isnull().sum())


mapped_priorities = detroit_df.groupby(by = ['priority'])['calldescription'].apply(lambda x: ', '.join(x)).reset_index()
mapped_priorities['calldescription'] = mapped_priorities['calldescription'].apply(lambda x: ", ".join(_.strip() for _ in set(x.split(","))))

### Fix Missing Priorities

In [4]:
priority_dict = {}
for priority in mapped_priorities['priority'].unique():
    priority_dict[priority] = mapped_priorities[(mapped_priorities.priority == priority)]['calldescription'].values.tolist()[0]

del priority_dict[" "]

def get_priority(current_priority, call_descrip):
    if current_priority != " ":
        return current_priority
    for key in priority_dict.keys():
        if call_descrip in priority_dict[key]:
            return key
        
    return 'FAIL'

detroit_df['new_priority'] = detroit_df.apply(lambda row: get_priority(row.priority, row.calldescription), axis = 1)

detroit_df.drop(columns = ['priority'], inplace = True)
detroit_df.rename(columns = {'new_priority': 'priority'}, inplace = True)


In [5]:
# for column in detroit_df.columns:
#     print(f'{column}\nNumber of Values: {len(detroit_df[column].unique())}\n')

In [6]:
detroit_df.drop(columns = ['totaltime','totalresponsetime'], inplace = True)


### Convert timestamp to weekday

In [7]:
def return_weekday(time_stamp):
    time = dt.datetime.strptime(time_stamp, '%Y/%m/%d %H:%M:%S+00').date()
    return dt.datetime.strftime(time, '%A')
    
detroit_df['weekday'] = detroit_df['call_timestamp'].apply(lambda x: return_weekday(x))

In [8]:
def get_daypart(time_stamp):
    time = dt.datetime.strptime(time_stamp, '%Y/%m/%d %H:%M:%S+00')
    time = dt.datetime.strftime(time, '%H:%M:%S')
    if time < "06:00":
        time_of_day = "Early Morning"
    elif time >= "06:00" and time < "10:00":
        time_of_day = "Morning"
    elif time >= "10:00" and time < "12:00":
        time_of_day = "Late Morning"
    elif time >= "12:00" and time < "15:00":
        time_of_day = "Afternoon"
    elif time >= "15:00" and time < "18:00":
        time_of_day = "Late Afternoon"
    elif time >= "18:00" and time < "21:00":
        time_of_day = "Early Evening"
    elif time >= "21:00":
        time_of_day = "Evening"
    else:
        time_of_day = "FAILED"
    return time_of_day

detroit_df['day_part'] = detroit_df['call_timestamp'].apply(lambda x: get_daypart(x))


In [9]:
detroit_df.day_part.unique()

array(['Morning', 'Late Morning', 'Afternoon', 'Late Afternoon',
       'Early Evening', 'Evening', 'Early Morning'], dtype=object)

In [10]:
detroit_df.head()

,callcode,calldescription,call_timestamp,precinct_sca,officerinitiated,intaketime,dispatchtime,traveltime,time_on_scene,neighborhood,council_district,priority,weekday,day_part
0,825030,DISTURBANCE,2016/09/20 09:36:12+00,910A,No,2.2,528.6,15.5,120.5,Wade,4.0,3,Tuesday,Morning
1,811020,AUTO X UNK INJ / IMPAIRED,2016/09/20 09:46:32+00,1201,No,2.7,5.2,6.4,59.1,Tri-Point,2.0,2,Tuesday,Morning
2,374030,RECOVER AUTO,2016/09/20 09:48:24+00,123B,No,4.9,NaN,NaN,NaN,Oak Grove,2.0,3,Tuesday,Morning
3,825030,DISTURBANCE,2016/09/20 09:50:29+00,0207,No,1.4,3.4,7.6,41.7,We Care Community,7.0,3,Tuesday,Morning
4,374030,RECOVER AUTO,2016/09/20 09:54:37+00,28A,No,3.0,1645.3,NaN,NaN,Barton-McFarland,7.0,3,Tuesday,Morning


In [11]:
detroit_df.drop(columns = ['callcode','call_timestamp','precinct_sca'], inplace = True)
detroit_df

,calldescription,officerinitiated,intaketime,dispatchtime,traveltime,time_on_scene,neighborhood,council_district,priority,weekday,day_part
0,DISTURBANCE,No,2.2,528.6,15.5,120.5,Wade,4.0,3,Tuesday,Morning
1,AUTO X UNK INJ / IMPAIRED,No,2.7,5.2,6.4,59.1,Tri-Point,2.0,2,Tuesday,Morning
2,RECOVER AUTO,No,4.9,NaN,NaN,NaN,Oak Grove,2.0,3,Tuesday,Morning
3,DISTURBANCE,No,1.4,3.4,7.6,41.7,We Care Community,7.0,3,Tuesday,Morning
4,RECOVER AUTO,No,3.0,1645.3,NaN,NaN,Barton-McFarland,7.0,3,Tuesday,Morning
...,...,...,...,...,...,...,...,...,...,...,...
2551995,BREAKING AND ENTERING AUTO I/P,No,2.2,19.3,9.1,48.0,Oakman Blvd Community,2.0,3,Monday,Late Morning
2551996,ONE OVER THE WHEEL,No,1.9,NaN,NaN,NaN,North Corktown,6.0,2,Monday,Late Morning
2551997,LARCENY I/P OR J/H,No,1.1,20.6,5.1,152.7,Oakman Blvd Community,7.0,3,Monday,Late Morning
2551998,SHOT OR STABBED DELTA,No,0.9,0.3,7.7,213.3,Warrendale,7.0,1,Monday,Late Morning


In [24]:
detroit_df[(detroit_df[detroit_df.columns[10]] == ' ')]

,calldescription,officerinitiated,intaketime,dispatchtime,traveltime,time_on_scene,neighborhood,council_district,priority,weekday,day_part


In [31]:
detroit_df['on_scene_2_hours'] = detroit_df['time_on_scene'].apply(lambda x: 1 if x > 120 else 0)

In [33]:
detroit_df.drop(columns = ['time_on_scene'], inplace = True)

In [34]:
X = detroit_df.drop(columns = ['on_scene_2_hours']).copy()

In [83]:
y_res = detroit_df['on_scene_2_hours'].copy()

In [84]:
X.select_dtypes('object')

,calldescription,officerinitiated,neighborhood,priority,weekday,day_part
0,DISTURBANCE,No,Wade,3,Tuesday,Morning
1,AUTO X UNK INJ / IMPAIRED,No,Tri-Point,2,Tuesday,Morning
2,RECOVER AUTO,No,Oak Grove,3,Tuesday,Morning
3,DISTURBANCE,No,We Care Community,3,Tuesday,Morning
4,RECOVER AUTO,No,Barton-McFarland,3,Tuesday,Morning
...,...,...,...,...,...,...
2551995,BREAKING AND ENTERING AUTO I/P,No,Oakman Blvd Community,3,Monday,Late Morning
2551996,ONE OVER THE WHEEL,No,North Corktown,2,Monday,Late Morning
2551997,LARCENY I/P OR J/H,No,Oakman Blvd Community,3,Monday,Late Morning
2551998,SHOT OR STABBED DELTA,No,Warrendale,1,Monday,Late Morning


In [85]:
y_res.unique()

array([1, 0], dtype=int64)

In [64]:
X_encoded = pd.get_dummies(X)
# detroit_dummied.shape

# It is better to use One-Hot-Encoding than get_dummies because OHE saves the exploded categories into its object, but we are using get_dummies for ease



In [65]:
X_encoded.head()

,intaketime,dispatchtime,traveltime,council_district,calldescription_ABNORMAL BREATHING,calldescription_ADMIT OR E/E,calldescription_AID MORTORIST MISC,calldescription_AID MOTORIST / CHILD LOCKED IN,calldescription_AID MOTORIST CHILD LOCKED INSD,calldescription_AID MOTORIST MISC,...,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,day_part_Afternoon,day_part_Early Evening,day_part_Early Morning,day_part_Evening,day_part_Late Afternoon,day_part_Late Morning,day_part_Morning
0,2.2,528.6,15.5,4.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,2.7,5.2,6.4,2.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,4.9,NaN,NaN,2.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,1.4,3.4,7.6,7.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,3.0,1645.3,NaN,7.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [66]:
X_encoded.fillna(0, inplace = True)

In [86]:
print(sum(y_res)/len(y_res))

0.04584561128526646


In [39]:
y.head()

0    1
1    0
2    0
3    0
4    0
Name: on_scene_2_hours, dtype: int64

In [112]:
def check_dmatrix(X,y, size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

    clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=0, seed=42)
    clf_xgb.fit(X_train, 
                y_train, 
                verbose = True,
                early_stopping_rounds=10, 
                eval_metric='aucpr',
                eval_set=[(X_test, y_test)]
               )
    
    accuracy = clf_xgb.score(X_test, y_test)
    scores.append([size,accuracy])

In [113]:
X_encoded.isna().sum().sum()

0

In [ ]:
scores = []
sizes = [100,1000,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000] + [_ for _ in range(200000,2500000,100000)]

for size in sizes:
    # if size > 100000:
    #     break
    X_subset = X_encoded.head(size)
    y_subset = y_res.head(size)

    ## converting to numpy array
    # X_subset = X_subset.to_numpy()
    # y_subset = y_subset.to_numpy()
    
    # print(X_subset)
    check_dmatrix(X_subset,y_subset, size)
    

[0]	validation_0-aucpr:0.05000
[1]	validation_0-aucpr:0.05000
[2]	validation_0-aucpr:0.05000
[3]	validation_0-aucpr:0.05000
[4]	validation_0-aucpr:0.03735
[5]	validation_0-aucpr:0.07671
[6]	validation_0-aucpr:0.25000
[7]	validation_0-aucpr:0.33333
[8]	validation_0-aucpr:0.50000


C:\Users\Work\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[9]	validation_0-aucpr:1.00000
[10]	validation_0-aucpr:1.00000
[11]	validation_0-aucpr:1.00000
[12]	validation_0-aucpr:0.30685
[13]	validation_0-aucpr:0.30685
[14]	validation_0-aucpr:0.30685
[15]	validation_0-aucpr:0.30685
[16]	validation_0-aucpr:0.30685
[17]	validation_0-aucpr:0.30685
[18]	validation_0-aucpr:0.30685
[0]	validation_0-aucpr:0.08876
[1]	validation_0-aucpr:0.08458


C:\Users\Work\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[2]	validation_0-aucpr:0.08639
[3]	validation_0-aucpr:0.17020
[4]	validation_0-aucpr:0.16862
[5]	validation_0-aucpr:0.17610
[6]	validation_0-aucpr:0.17413
[7]	validation_0-aucpr:0.17877
[8]	validation_0-aucpr:0.17330
[9]	validation_0-aucpr:0.17356
[10]	validation_0-aucpr:0.17367
[11]	validation_0-aucpr:0.17579
[12]	validation_0-aucpr:0.18180
[13]	validation_0-aucpr:0.17907
[14]	validation_0-aucpr:0.18230
[15]	validation_0-aucpr:0.18357
[16]	validation_0-aucpr:0.18209
[17]	validation_0-aucpr:0.18491
[18]	validation_0-aucpr:0.18255
[19]	validation_0-aucpr:0.18314
[20]	validation_0-aucpr:0.10038
[21]	validation_0-aucpr:0.10196
[22]	validation_0-aucpr:0.19136
[23]	validation_0-aucpr:0.10749
[24]	validation_0-aucpr:0.09144
[25]	validation_0-aucpr:0.09158
[26]	validation_0-aucpr:0.09269
[27]	validation_0-aucpr:0.09202
[28]	validation_0-aucpr:0.09261
[29]	validation_0-aucpr:0.09194
[30]	validation_0-aucpr:0.09027
[31]	validation_0-aucpr:0.09077


...

C:\Users\Work\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-aucpr:0.19294
[1]	validation_0-aucpr:0.19451
[2]	validation_0-aucpr:0.20671
[3]	validation_0-aucpr:0.21618
[4]	validation_0-aucpr:0.21729
[5]	validation_0-aucpr:0.21780
[6]	validation_0-aucpr:0.22060
[7]	validation_0-aucpr:0.22450
[8]	validation_0-aucpr:0.22885
[9]	validation_0-aucpr:0.22928
[10]	validation_0-aucpr:0.23115
[11]	validation_0-aucpr:0.23355
[12]	validation_0-aucpr:0.23475
[13]	validation_0-aucpr:0.23710
[14]	validation_0-aucpr:0.23867
[15]	validation_0-aucpr:0.24012
[16]	validation_0-aucpr:0.24029
[17]	validation_0-aucpr:0.24153
[18]	validation_0-aucpr:0.24348
[19]	validation_0-aucpr:0.24415
[20]	validation_0-aucpr:0.24520
[21]	validation_0-aucpr:0.24529
[22]	validation_0-aucpr:0.24677
[23]	validation_0-aucpr:0.24691
[24]	validation_0-aucpr:0.24757
[25]	validation_0-aucpr:0.24776
[26]	validation_0-aucpr:0.24813
[27]	validation_0-aucpr:0.24819
[28]	validation_0-aucpr:0.24904
[29]	validation_0-aucpr:0.24927
[30]	validation_0-aucpr:0.24941
[31]	validation_0-

C:\Users\Work\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-aucpr:0.19377
[1]	validation_0-aucpr:0.19477
[2]	validation_0-aucpr:0.20626
[3]	validation_0-aucpr:0.21409
[4]	validation_0-aucpr:0.21484
[5]	validation_0-aucpr:0.21589
[6]	validation_0-aucpr:0.21780
[7]	validation_0-aucpr:0.22179
[8]	validation_0-aucpr:0.22373
[9]	validation_0-aucpr:0.22433
[10]	validation_0-aucpr:0.22889
[11]	validation_0-aucpr:0.22858
[12]	validation_0-aucpr:0.23015
[13]	validation_0-aucpr:0.23067
[14]	validation_0-aucpr:0.23248
[15]	validation_0-aucpr:0.23254
[16]	validation_0-aucpr:0.23460
[17]	validation_0-aucpr:0.23640
[18]	validation_0-aucpr:0.23798
[19]	validation_0-aucpr:0.23993
[20]	validation_0-aucpr:0.24019
[21]	validation_0-aucpr:0.24128
[22]	validation_0-aucpr:0.24269
[23]	validation_0-aucpr:0.24280
[24]	validation_0-aucpr:0.24335
[25]	validation_0-aucpr:0.24400
[26]	validation_0-aucpr:0.24419
[27]	validation_0-aucpr:0.24497
[28]	validation_0-aucpr:0.24503
[29]	validation_0-aucpr:0.24592
[30]	validation_0-aucpr:0.24604
[31]	validation_0-

C:\Users\Work\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns


score_frame = pd.DataFrame(scores, columns = ['size','accuracy'])

plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13

fig = plt.figure(figsize = (12,5))
plt.title("Accuracy vs. Number of Observations", fontsize = 22)

ax = sns.scatterplot(data = score_frame, x = 'size', y = 'accuracy', s = 180, alpha = .8, edgecolor = 'black', color = 'red')
plt.axhline(0, color = 'black', linestyle = 'dashed')

ax.set_xlabel('Number of Observations', fontsize = 15)
ax.set_ylabel('Accuracy', fontsize = 15)
ax.tick_params(size = 10)
# ax.set_xlim([-1000,2500000])
ax.set_ylim([.93,.97])
ax.ticklabel_format(style = 'plain')
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
sns.despine()


In [ ]:
detroit_df.shape

In [ ]:
bst = clf_xgb.get_booster()
for importance_type in ('weight','gain','')

xgb.to_graphviz(clf_xgb,
                num_trees=0,
                size="10,10"
                condition_node_params=node_parmas,
                leaf_node_params=leaf_params
               )